# Practical 1a: Saprk Basic I

This notebook is the end-to-end example, showing how to use DataFrame and Spark SQL for common data analytics patterns and operations on a [San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3) dataset.

Source: https://github.com/databricks/LearningSparkV2

Inspect location where the SF Fire Department Fire calls data set is stored in the public dataset S3 bucket

In [0]:
%fs ls /databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

path,name,size,modificationTime
dbfs:/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv,sf-fire-calls.csv,1137925359,1576280979000


Define the location of the public dataset on the S3 bucket

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

Inspect the data looks like before defining a schema

In [0]:
%fs head databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

[Truncated to first 65536 bytes]
Call Number,Unit ID,Incident Number,CallType,Call Date,Watch Date,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,OrigPriority,Priority,Final Priority,ALS Unit,Call Type Group,NumAlarms,UnitType,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,"",1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666
20110016,E03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,7,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E03,2.683333333333333
20110016,E38,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:17 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,1,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E38,2.1
20110016,E41,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,8,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E41,2.716666666666667
20110016,M03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:38 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,true,"",1,MEDIC,10,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-M03,2.7666666666666666
20110016,RS1,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:57 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,RESCUE SQUAD,9,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-RS1,3.2666666666666666
20110016,T03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,5,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T03,3.2333333333333334
20110016,T13,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:44 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,2,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T13,2.95
20110017,M13,2003236,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:27:14 AM,700 Block of FOLSOM ST,SF,94107,B03,01,2176,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7831757914875, -122.399603541822)",020110017-M13,6.933333333333334
20110019,M36,2003238,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:28:30 AM,9TH ST/LARKIN ST,SF,94103,B02,36,2337,1,1,2,true,"",1,MEDIC,1,2,6,South of Market,"(37.7774938436091, -122.41629170277)",020110019-M36,6.116666666666666
20110020,E38,2003239,Outside Fire,01/11/2002,01/10/2002,Other,01/11/2002 02:15:33 AM,3000 Block of WEBSTER ST,SF,94123,B04,16,3462,3,

Define our schema as the file has 4 million records. Inferring the schema is expensive for large files.

In [0]:
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

In [0]:
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

Cache the DataFrame since we will be performing some operations on it.

In [0]:
fire_df.cache()

Out[5]: DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: string, WatchDate: string, CallFinalDisposition: string, AvailableDtTm: string, Address: string, City: string, Zipcode: int, Battalion: string, StationArea: string, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumAlarms: int, UnitType: string, UnitSequenceInCallDispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, Neighborhood: string, Location: string, RowID: string, Delay: float]

In [0]:
fire_df.count()

Out[6]: 4380660

In [0]:
fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [0]:
display(fire_df.limit(5))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.3166666
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166667


Filter out "Medical Incident" call types

Note that `filter()` and `where()` methods on the DataFrame are similar. Check relevant documentation for their respective argument types.

In [0]:
few_fire_df = (fire_df
               .select("IncidentNumber", "AvailableDtTm", "CallType")
               .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:47:00 AM|Structure Fire|
|2003235       |01/11/2002 01:51:54 AM|Structure Fire|
|2003235       |01/11/2002 01:47:00 AM|Structure Fire|
|2003235       |01/11/2002 01:47:00 AM|Structure Fire|
|2003235       |01/11/2002 01:51:17 AM|Structure Fire|
+--------------+----------------------+--------------+
only showing top 5 rows



**Q-1) How many distinct types of calls were made to the Fire Department?**

To be sure, let's not count "null" strings in that column.

In [0]:
fire_df.select("CallType").where(col("CallType").isNotNull()).distinct().count()

Out[10]: 32

**Q-2) What are distinct types of calls were made to the Fire Department?**

These are all the distinct type of call to the SF Fire Department

In [0]:
fire_df.select("CallType").where(col("CallType").isNotNull()).distinct().show(10, False)

+-----------------------------+
|CallType                     |
+-----------------------------+
|Alarms                       |
|Odor (Strange / Unknown)     |
|Citizen Assist / Service Call|
|Vehicle Fire                 |
|Other                        |
|Outside Fire                 |
|Electrical Hazard            |
|Structure Fire               |
|Medical Incident             |
|Fuel Spill                   |
+-----------------------------+
only showing top 10 rows



**Q-3) Find out all response or delayed times greater than 5 mins?**

1. Rename the column Delay - > ReponseDelayedinMins
2. Returns a new DataFrame
3. Find out all calls where the response time to the fire site was delayed for more than 5 mins

In [0]:
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
new_fire_df.select("ResponseDelayedinMins").where(col("ResponseDelayedinMins") > 5).show(5, False)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.233333             |
|6.9333334            |
|6.116667             |
|7.85                 |
|77.333336            |
+---------------------+
only showing top 5 rows



Let's do some ETL:

1. Transform the string dates to Spark Timestamp data type so we can make some time-based queries later
2. Returns a transformed query
3. Cache the new DataFrame

In [0]:
fire_ts_df = (new_fire_df
              .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
              .withColumn("OnWatchDate",   to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate")
              .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm"))          

In [0]:
fire_ts_df.cache()
fire_ts_df.columns

Out[14]: ['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallFinalDisposition',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'ResponseDelayedinMins',
 'IncidentDate',
 'OnWatchDate',
 'AvailableDtTS']

Check the transformed columns with Spark Timestamp type

In [0]:
fire_ts_df.select("IncidentDate", "OnWatchDate", "AvailableDtTS").show(5, False)

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:58:43|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:10:17|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:54|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



**Q-4) What were the most common call types?**

List them in descending order

In [0]:
(fire_ts_df
 .select("CallType").where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count()
 .orderBy("count", ascending=False)
 .show(n=10, truncate=False))

+-------------------------------+-------+
|CallType                       |count  |
+-------------------------------+-------+
|Medical Incident               |2843475|
|Structure Fire                 |578998 |
|Alarms                         |483518 |
|Traffic Collision              |175507 |
|Citizen Assist / Service Call  |65360  |
|Other                          |56961  |
|Outside Fire                   |51603  |
|Vehicle Fire                   |20939  |
|Water Rescue                   |20037  |
|Gas Leak (Natural and LP Gases)|17284  |
+-------------------------------+-------+
only showing top 10 rows



**Q-4a) What zip codes accounted for most common calls?**

Let's investigate what zip codes in San Francisco accounted for most fire calls and what type where they.

1. Filter out by CallType
2. Group them by CallType and Zip code
3. Count them and display them in descending order

It seems like the most common calls were all related to Medical Incident, and the two zip codes are 94102 and 94103.

In [0]:
(fire_ts_df
 .select("CallType", "ZipCode")
 .where(col("CallType").isNotNull())
 .groupBy("CallType", "Zipcode")
 .count()
 .orderBy("count", ascending=False)
 .show(10, truncate=False))

+----------------+-------+------+
|CallType        |Zipcode|count |
+----------------+-------+------+
|Medical Incident|94102  |401457|
|Medical Incident|94103  |370215|
|Medical Incident|94110  |249279|
|Medical Incident|94109  |238087|
|Medical Incident|94124  |147564|
|Medical Incident|94112  |139565|
|Medical Incident|94115  |120087|
|Medical Incident|94122  |107602|
|Medical Incident|94107  |107439|
|Medical Incident|94133  |99050 |
+----------------+-------+------+
only showing top 10 rows



**Q-4b) What San Francisco neighborhoods are in the zip codes 94102 and 94103**

Let's find out the neighborhoods associated with these two zip codes. In all likelihood, these are some of the contested 
neighborhood with high reported crimes.

In [0]:
fire_ts_df.select("Neighborhood", "Zipcode").where((col("Zipcode") == 94102) | (col("Zipcode") == 94103)).distinct().show(10, truncate=False)

+------------------------------+-------+
|Neighborhood                  |Zipcode|
+------------------------------+-------+
|Western Addition              |94102  |
|Tenderloin                    |94102  |
|Nob Hill                      |94102  |
|Castro/Upper Market           |94103  |
|South of Market               |94102  |
|South of Market               |94103  |
|Financial District/South Beach|94102  |
|Tenderloin                    |94103  |
|Financial District/South Beach|94103  |
|Hayes Valley                  |94102  |
+------------------------------+-------+
only showing top 10 rows



**Q-5) What was the sum of all calls, average, min and max of the response times for calls?**

Let's use the built-in Spark SQL functions to compute the sum, avg, min, and max of few columns:

* Number of Total Alarms
* What were the min and max the delay in response time before the Fire Dept arrived at the scene of the call

In [0]:
fire_ts_df.select(sum("NumAlarms"), avg("ResponseDelayedinMins"), min("ResponseDelayedinMins"), max("ResponseDelayedinMins")).show()

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|       4403441|         3.902170335891614|               0.016666668|                 1879.6167|
+--------------+--------------------------+--------------------------+--------------------------+



** Q-6a) How many distinct years of data is in the CSV file?**

We can use the `year()` SQL Spark function off the Timestamp column data type IncidentDate.

In all, we have fire calls from years 2000-2018

In [0]:
fire_ts_df.select(year("IncidentDate")).distinct().orderBy(year("IncidentDate")).show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



** Q-6b) What week of the year in 2018 had the most fire calls?**

**Note**: Week 1 is the New Years' week and week 25 is the July 4 the week. Loads of fireworks, so it makes sense the higher number of calls.

In [0]:
fire_ts_df.filter(year("IncidentDate") == 2018).groupBy(weekofyear("IncidentDate")).count().orderBy("count", ascending=False).show()

+------------------------+-----+
|weekofyear(IncidentDate)|count|
+------------------------+-----+
|                       1| 6401|
|                      25| 6163|
|                      13| 6103|
|                      22| 6060|
|                      44| 6048|
|                      27| 6042|
|                      16| 6009|
|                      40| 6000|
|                      43| 5986|
|                       5| 5946|
|                       2| 5929|
|                      18| 5917|
|                       9| 5874|
|                       8| 5843|
|                       6| 5839|
|                      21| 5821|
|                      38| 5817|
|                      10| 5806|
|                      23| 5781|
|                      32| 5764|
+------------------------+-----+
only showing top 20 rows



** Q-7) What neighborhoods in San Francisco had the worst response time in 2018?**

It appears that if you living in Presidio Heights, the Fire Dept arrived in less than 3 mins, while Mission Bay took more than 6 mins.

In [0]:
fire_ts_df.select("Neighborhood", "ResponseDelayedinMins").filter(year("IncidentDate") == 2018).show(10, False)

+----------------+---------------------+
|Neighborhood    |ResponseDelayedinMins|
+----------------+---------------------+
|Presidio Heights|2.4666667            |
|Presidio Heights|2.8833334            |
|Presidio Heights|2.1166666            |
|Presidio Heights|4.25                 |
|Presidio Heights|2.8166666            |
|Presidio Heights|3.4                  |
|Presidio Heights|2.7333333            |
|Mission Bay     |6.266667             |
|Mission Bay     |6.733333             |
|Mission Bay     |6.3333335            |
+----------------+---------------------+
only showing top 10 rows



** Q-8a) How can we use Parquet files or SQL table to store data and read it back?**

In [0]:
fire_ts_df.write.format("parquet").mode("overwrite").save("/tmp/fireServiceParquet/")

In [0]:
%fs ls /tmp/fireServiceParquet/

path,name,size,modificationTime
dbfs:/tmp/fireServiceParquet/_SUCCESS,_SUCCESS,0,1676011577000
dbfs:/tmp/fireServiceParquet/_committed_3682280538891507627,_committed_3682280538891507627,924,1672907543000
dbfs:/tmp/fireServiceParquet/_committed_8186096185729529859,_committed_8186096185729529859,1834,1676011576000
dbfs:/tmp/fireServiceParquet/_committed_vacuum4991261393602229716,_committed_vacuum4991261393602229716,96,1676011578000
dbfs:/tmp/fireServiceParquet/_started_8186096185729529859,_started_8186096185729529859,0,1676011509000
dbfs:/tmp/fireServiceParquet/part-00000-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-226-1-c000.snappy.parquet,part-00000-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-226-1-c000.snappy.parquet,19478331,1676011569000
dbfs:/tmp/fireServiceParquet/part-00001-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-227-1-c000.snappy.parquet,part-00001-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-227-1-c000.snappy.parquet,19690958,1676011569000
dbfs:/tmp/fireServiceParquet/part-00002-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-228-1-c000.snappy.parquet,part-00002-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-228-1-c000.snappy.parquet,19317318,1676011569000
dbfs:/tmp/fireServiceParquet/part-00003-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-229-1-c000.snappy.parquet,part-00003-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-229-1-c000.snappy.parquet,17794007,1676011569000
dbfs:/tmp/fireServiceParquet/part-00004-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-230-1-c000.snappy.parquet,part-00004-tid-8186096185729529859-3c97f9aa-4263-4fd5-b594-f9c551b934a1-230-1-c000.snappy.parquet,20768695,1676011569000


** Q-8b) How can read data from Parquet file?**

Note we don't have to specify the schema here since it's stored as part of the Parquet metadata

In [0]:
file_parquet_df = spark.read.format("parquet").load("/tmp/fireServiceParquet/")

In [0]:
display(file_parquet_df.limit(10))

CallNumber,UnitID,IncidentNumber,CallType,CallFinalDisposition,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,ResponseDelayedinMins,IncidentDate,OnWatchDate,AvailableDtTS
111050354,E14,11034920,Medical Incident,Other,500 Block of 21ST AVE,SF,94121,B07,14,7171,3,3,3,true,null,1,ENGINE,1,7,1,Outer Richmond,"(37.7774255992901, -122.480311994328)",111050354-E14,4.7833333,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:27:08.000+0000
111050355,E03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,true,null,1,ENGINE,1,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-E03,1.9166666,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:10:54.000+0000
111050355,T03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,false,null,1,TRUCK,2,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-T03,2.4333334,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:10:54.000+0000
111050356,73,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,MEDIC,10,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-73,2.0666666,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:24:56.000+0000
111050356,B06,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,6,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B06,2.6,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:22:46.000+0000
111050356,B10,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,4,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B10,3.25,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:25:00.000+0000
111050356,D3,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,7,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-D3,3.5,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:23:01.000+0000
111050356,E29,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,ENGINE,8,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E29,2.6,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:22:50.000+0000
111050356,E37,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,ENGINE,2,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E37,2.6666667,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:25:10.000+0000
111050356,RS2,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,RESCUE SQUAD,5,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-RS2,3.05,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:24:11.000+0000
